# 12 - Control Unit

## Introduction

The **Control Unit** is the brain of the CPU. It takes the decoded instruction and generates all the control signals needed to execute it.

The Control Unit implements a **state machine** that cycles through:
1. **FETCH** - Get instruction from memory
2. **DECODE** - Understand the instruction
3. **EXECUTE** - Perform the operation
4. **WRITEBACK** - Store results

## Learning Objectives

1. Understand the CPU state machine
2. Learn how to generate control signals
3. Build the Control Unit

In [ ]:
import sys
from pathlib import Path
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root / 'src'))
sys.path.insert(0, str(project_root))

from computer.clock import ControlSignals
from computer.isa import OPCODES
from typing import Dict
print("Setup complete!")

## CPU State Machine

```mermaid
stateDiagram-v2
    [*] --> FETCH
    FETCH --> DECODE
    DECODE --> EXECUTE
    EXECUTE --> WRITEBACK
    WRITEBACK --> FETCH
```

Each state generates different control signals.

## Exercise 1: Control Unit

In [ ]:
class ControlUnit:
    """CPU Control Unit - generates control signals based on state and instruction."""
    
    # State constants
    FETCH = 'FETCH'
    DECODE = 'DECODE'
    EXECUTE = 'EXECUTE'
    WRITEBACK = 'WRITEBACK'
    
    def __init__(self):
        self.state = self.FETCH
        self.signals = ControlSignals()
    
    def generate_signals(self, decoded: Dict, flags: Dict) -> ControlSignals:
        """
        Generate control signals based on current state and decoded instruction.
        
        Args:
            decoded: Decoded instruction (from InstructionDecoder)
            flags: CPU flags {'Z': 0/1, 'C': 0/1, 'N': 0/1, 'V': 0/1}
        
        Returns:
            ControlSignals for current state
        """
        self.signals.reset()
        opcode = decoded.get('opcode', 0)
        opname = decoded.get('opcode_name', 'NOP')
        
        # YOUR CODE HERE
        # Generate signals based on self.state and opname
        # 
        # FETCH: mem_read=1, ir_load=1
        # DECODE: (no signals needed)
        # EXECUTE: depends on instruction type
        # WRITEBACK: pc_inc=1 (usually)
        pass
    
    def next_state(self) -> str:
        """
        Advance to next state.
        
        Returns:
            New state name
        """
        # YOUR CODE HERE
        # FETCH -> DECODE -> EXECUTE -> WRITEBACK -> FETCH
        pass
    
    def reset(self) -> None:
        """Reset to initial state."""
        self.state = self.FETCH
        self.signals.reset()

# Test
cu = ControlUnit()
decoded = {'opcode': OPCODES['ADD'], 'opcode_name': 'ADD', 'rd': 0, 'rs1': 1, 'rs2_imm': 2}
flags = {'Z': 0, 'C': 0, 'N': 0, 'V': 0}

print("State Machine Test:")
for _ in range(8):  # Two complete cycles
    print(f"  State: {cu.state:10}", end="")
    signals = cu.generate_signals(decoded, flags)
    active = [k for k, v in signals.to_dict().items() if v != 0 and v != [0,0,0,0]]
    print(f"  Active: {active}")
    cu.next_state()

## Control Signal Generation for Different Instructions

Let's see what signals each instruction type needs during EXECUTE:

In [ ]:
def show_execute_signals(opname: str):
    """Show what signals an instruction generates in EXECUTE state."""
    cu = ControlUnit()
    cu.state = cu.EXECUTE
    
    decoded = {'opcode': OPCODES[opname], 'opcode_name': opname}
    flags = {'Z': 0, 'C': 0, 'N': 0, 'V': 0}
    
    signals = cu.generate_signals(decoded, flags)
    
    print(f"\n{opname}:")
    for name, value in signals.to_dict().items():
        if value != 0 and value != [0,0,0,0]:
            print(f"  {name} = {value}")

# Show execute signals for each instruction type
for instr in ['ADD', 'LOAD', 'STORE', 'JMP', 'HALT']:
    show_execute_signals(instr)

## Conditional Jumps

JZ and JNZ need to check the Zero flag:

In [ ]:
def test_conditional_jump(opname: str, zero_flag: int):
    """Test conditional jump with given Zero flag."""
    cu = ControlUnit()
    cu.state = cu.EXECUTE
    
    decoded = {'opcode': OPCODES[opname], 'opcode_name': opname, 'rs2_imm': 0x10}
    flags = {'Z': zero_flag, 'C': 0, 'N': 0, 'V': 0}
    
    signals = cu.generate_signals(decoded, flags)
    
    print(f"\n{opname} with Z={zero_flag}:")
    if signals.pc_load:
        print("  -> JUMP TAKEN (pc_load=1)")
    elif signals.pc_inc:
        print("  -> JUMP NOT TAKEN (pc_inc=1)")
    else:
        print("  -> NO PC CHANGE")

# JZ: Jump if Zero
test_conditional_jump('JZ', 0)  # Z=0: don't jump
test_conditional_jump('JZ', 1)  # Z=1: jump

# JNZ: Jump if Not Zero
test_conditional_jump('JNZ', 0)  # Z=0: jump
test_conditional_jump('JNZ', 1)  # Z=1: don't jump

## Copy Your Implementation

Once your code works, copy the `ControlUnit` class to `src/computer/control.py`

## Validation

In [ ]:
from utils.checker import check
check('control')

## Summary

The Control Unit:

| State | Purpose | Key Signals |
|-------|---------|-------------|
| FETCH | Get instruction | mem_read, ir_load |
| DECODE | Parse instruction | (none) |
| EXECUTE | Perform operation | alu_op, mem_read/write |
| WRITEBACK | Store result | reg_write, pc_inc |

### Phase 3 Complete!

We now have all the control components:
- Clock for timing
- ISA defining our instructions
- Decoder to parse instructions
- Control Unit to generate signals

### What's Next?

In Phase 4, we'll connect everything together:
- **Data Path** - Wire up all components
- **CPU** - Fetch-decode-execute cycle
- **Assembler** - Convert assembly to machine code
- **Full System** - Run real programs!